In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception

from tensorflow.keras.applications import NASNetLarge, DenseNet201

from tensorflow.keras.models import Sequential
from glob import glob
import matplotlib.pyplot as plt
# from tensorflow.keras.applications import NASNetLarge

from tensorflow.keras  import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D,BatchNormalization
from tensorflow.keras import layers

import os 
import pandas as pd 
import numpy as np 


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.models import load_model

import tensorflow as tf
from tensorflow.python.client import device_lib

from tensorflow import keras

In [ ]:
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5621387722606899803
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3703371726706229486
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4059937463847348210
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15695549568
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13052639706118451287
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import zipfile

with zipfile.ZipFile('/content/gdrive/My Drive/Butterfly/butterfly_classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/gdrive/My Drive/Butterfly')

In [ ]:
ls '/content/gdrive/My Drive/Butterfly/DATA'

TEST/  TRAIN/


In [ ]:
data_dir = '/content/gdrive/My Drive/Butterfly/DATA'
train_dir = os.path.join(data_dir,"TRAIN")
test_dir = os.path.join(data_dir,"TEST")
print(train_dir)
print(test_dir)

/content/gdrive/My Drive/Butterfly/DATA/TRAIN
/content/gdrive/My Drive/Butterfly/DATA/TEST


In [ ]:
IMAGE_SIZE = (240, 240,3)

# add preprocessing layer to the front of VGG
xception = Xception(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)

for xception_layer in xception.layers:
  xception_layer.trainable = False


# add fully-connected & dropout layers
x = Dense(512,activation='relu')(xception.output)

x = Dense(256,activation='relu')(x)

x = Dense(128,activation='relu')(x)

x = Dense(64,activation='relu')(x)

x = Dense(34,activation='relu')(x)

x = Flatten()(x)
prediction_layer = Dense(50, activation='softmax')(x)

# create a model object
model = Model(inputs=xception.input, outputs=prediction_layer)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss=tf.keras.losses.CategoricalCrossentropy(),
  optimizer=tf.keras.optimizers.Adam(),
  metrics=['accuracy']
)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 240, 240, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 119, 119, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 119, 119, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 119, 119, 32) 0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________

In [ ]:
train_datagen = ImageDataGenerator(
                                    shear_range = 0.15,
                                   zoom_range = 0.15,
                                   horizontal_flip = True,
                                    rescale=1.0/255.0
)


training_set = train_datagen.flow_from_directory(train_dir,
                                         target_size = (240, 240),  
                                         color_mode='rgb',        
                                        class_mode='categorical', 
                                        batch_size=128, 
                                        )


Found 4479 images belonging to 50 classes.


In [ ]:
my_callbacks = [
    # https://keras.io/api/callbacks/model_checkpoint/
    ModelCheckpoint(filepath='/content/gdrive/My Drive/Butterfly/XceptionRethinkUX.ckpt',save_weights_only=True, verbose = 1),
    # https://keras.io/api/callbacks/reduce_lr_on_plateau/
    ReduceLROnPlateau(monitor="accuracy",factor=0.1,patience=2,verbose=1,mode="auto")
]

result = model.fit(
  training_set,
  batch_size=128,
  epochs=50,
  callbacks=my_callbacks,
)


Epoch 1/50
35/35 [==============================] - ETA: 0s - loss: 2.4487 - accuracy: 0.3273
Epoch 00001: saving model to /content/gdrive/My Drive/Butterfly/XceptionRethinkUX.ckpt
35/35 [==============================] - 79s 2s/step - loss: 2.4487 - accuracy: 0.3273
Epoch 2/50
35/35 [==============================] - ETA: 0s - loss: 1.1429 - accuracy: 0.6419
Epoch 00002: saving model to /content/gdrive/My Drive/Butterfly/XceptionRethinkUX.ckpt
35/35 [==============================] - 79s 2s/step - loss: 1.1429 - accuracy: 0.6419
Epoch 3/50
35/35 [==============================] - ETA: 0s - loss: 0.7666 - accuracy: 0.7508
Epoch 00003: saving model to /content/gdrive/My Drive/Butterfly/XceptionRethinkUX.ckpt
35/35 [==============================] - 79s 2s/step - loss: 0.7666 - accuracy: 0.7508
Epoch 4/50
35/35 [==============================] - ETA: 0s - loss: 0.6053 - accuracy: 0.8058
Epoch 00004: saving model to /content/gdrive/My Drive/Butterfly/XceptionRethinkUX.ckpt
35/35 [========

In [ ]:
 targets = os.listdir('/content/gdrive/My Drive/Butterfly/DATA/TRAIN')
targets.sort()

In [ ]:
data = np.zeros((np.shape(os.listdir("/content/gdrive/My Drive/Butterfly/DATA/TEST"))[0],2))
sub = pd.DataFrame(data,columns=['Filename','Lables'])
for i,file_n in enumerate(os.listdir("/content/gdrive/My Drive/Butterfly/DATA/TEST/")):
  img = keras.preprocessing.image.load_img(r"/content/gdrive/My Drive/Butterfly/DATA/TEST/"+file_n, target_size=(240,240))
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array /= 255.0

  img_array = tf.expand_dims(img_array, axis=0)  

  predictions = model.predict(img_array)
    
  pred = np.argmax(predictions,axis=1)

  if pred == 0:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[0]
  elif pred == 1:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[1]
  elif pred == 2:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[2]
  elif pred == 3:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[3]
  elif pred == 4:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[4]
  elif pred == 5:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[5]
  if pred == 6:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[6]
  elif pred == 7:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[7]
  elif pred == 8:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] =  targets[8]
  elif pred == 9:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[9]
  elif pred == 10:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[10]
  elif pred == 11:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[11]
  elif pred == 12:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[12]
  elif pred == 13:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[13]
  elif pred == 14:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[14]
  elif pred == 15:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[15]
  elif pred == 16:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[16]
  elif pred == 17:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[17]
  elif pred == 18:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[18]
  elif pred == 19:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[19]
  elif pred == 20:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[20]
  elif pred == 21:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[21]
  elif pred == 22:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[22]
  elif pred == 23:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[23]
  elif pred == 24:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[24]
  elif pred == 25:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[25]
  elif pred == 26:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[26]
  elif pred == 27:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[27]
  elif pred == 28:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[28]
  elif pred == 29:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[29]
  elif pred == 30:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[30]
  elif pred == 31:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[31]
  elif pred == 32:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[32]
  elif pred == 33:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[33]
  elif pred == 34:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[34]
  elif pred == 35:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[35]
  elif pred == 36:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[36]
  elif pred == 37:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[37]
  elif pred == 38:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[38]
  elif pred == 39:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[39]
  elif pred == 40:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[40]
  elif pred == 41:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[41]
  elif pred == 42:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[42]
  elif pred == 43:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[43]
  elif pred == 44:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[44]
  elif pred == 45:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[45]
  elif pred == 46:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[46]
  elif pred == 47:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[47]
  elif pred == 48:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[48]
  elif pred == 49:
    sub.iloc[i,sub.columns.get_loc('Filename')] = file_n
    sub.iloc[i,sub.columns.get_loc('Lables')] = targets[49]

In [ ]:
sub = sub.sort_values('Filename')
sub

,Filename,Lables
429,000.jpg,red admiral
469,001.jpg,straited queen
462,002.jpg,metalmark
59,003.jpg,pipevine swallow
174,004.jpg,scarce swallow
...,...,...
478,495.jpg,american snoot
292,496.jpg,red spotted purple
472,497.jpg,paper kite
159,498.jpg,pine white


In [ ]:
sub.to_csv("/content/gdrive/My Drive/Butterfly/submission1.csv",index=False)
